In [1]:
import timeit
import numba
from numba import cuda, int32
import numpy as np

In [2]:
@cuda.jit
def cuda_points_count(points, count):
    """
    CUDA-ядро для подсчета количества точек, попадающих в единичную окружность.

    Args:
        points: Массив точек (n x 2).
        count: Глобальный счетчик для хранения количества точек внутри окружности.
    """
    idx = cuda.grid(1)  # Глобальный индекс потока
    if idx < points.shape[0]:  # Проверка границ массива
        x, y = points[idx]
        if x ** 2 + y ** 2 <= 1.0:
            cuda.atomic.add(count, 0, 1)

def cpu_points_count(points: np.ndarray) -> int:
    """
    Подсчет количества точек, попадающих в единичную окружность на CPU.

    Args:
        points: Массив точек (n x 2).

    Returns:
        Количество точек внутри окружности.
    """
    return np.sum(np.sum(points ** 2, axis=1) <= 1)

def run_cuda_points_count(points: np.ndarray) -> int:
    """
    Запуск подсчета точек внутри окружности с использованием CUDA.

    Args:
        points: Массив точек (n x 2).

    Returns:
        Количество точек внутри окружности.
    """
    d_points = cuda.to_device(points)
    d_count = cuda.to_device(np.zeros(1, dtype=np.int32))  # Счетчик на устройстве

    threads_per_block = 256
    blocks_per_grid = (points.shape[0] + threads_per_block - 1) // threads_per_block

    # Запуск CUDA-ядра
    cuda_points_count[blocks_per_grid, threads_per_block](d_points, d_count)
    return d_count.copy_to_host()[0]


In [3]:
n = 10 ** 6  # Количество точек
number_of_tests = 12
points = (np.random.rand(n, 2) - 0.5) * 2  # Случайные точки в диапазоне [-1, 1] x [-1, 1]

cuda_result = 4 * run_cuda_points_count(points) / n
cpu_result = 4 * cpu_points_count(points) / n

time_cuda = timeit.timeit(lambda: run_cuda_points_count(points), number=number_of_tests) / number_of_tests
time_cpu = timeit.timeit(lambda: cpu_points_count(points), number=number_of_tests) / number_of_tests

# Вывод результатов
print(f"TIME CUDA = {time_cuda:.6f} s")
print(f"TIME CPU = {time_cpu:.6f} s")
print(f"SPEED UP = {time_cpu / time_cuda:.2f}x")
print(f"GPU: pi ≈ {cuda_result:.6f}")
print(f"CPU: pi ≈ {cpu_result:.6f}")


TIME CUDA = 0.005527 s
TIME CPU = 0.024369 s
SPEED UP = 4.41x
GPU: pi ≈ 3.141240
CPU: pi ≈ 3.141240
